 Raytracing!

In [1]:
import torch 
import matplotlib.pyplot as plt 
import einops
import itertools
import numpy as np 
import math
from dataclasses import dataclass

from icosphere import icosphere

In [2]:
@dataclass
class Environment:
    """ 
    An environment containing spheres and triangles 
    """
    sphere_positions: torch.Tensor
    sphere_radii: torch.Tensor
    triangles: torch.Tensor
    object_idxs: torch.Tensor
    device: str = 'cpu'
    dtype: torch.dtype = torch.float64

    def __post_init__(self):
        assert self.sphere_positions.shape[0]==self.sphere_radii.shape[0]

        self.to(device=self.device, dtype=self.dtype)
    
    def to(self, device, dtype):
        self.sphere_positions = self.sphere_positions.to(dtype=dtype, device=device)
        self.sphere_radii = self.sphere_radii.to(dtype=dtype, device=device)
        self.triangles = self.triangles.to(dtype=dtype, device=device)
        self.object_idxs = self.object_idxs.to(dtype=torch.int32, device=device)

        self.device = device 
        self.type=dtype

        return self

In [3]:
def give_wall_idx(vertices):
    """Which wall is this?
    return the index of the wall

    vertices is a tensor of shape (N, 3 vectors, 3 dims)
    """
    N, _, _ = vertices.shape
    # which axis all match
    all_zeros = (vertices.sum(-2)==0)*1
    all_ones = (vertices.sum(-2)==3)*1

    output = all_ones.any(-1)*3 + all_zeros.argmax(-1) + all_ones.argmax(-1)

    return output

def make_room():
    # now create the environment
    central_vertices = [
        [0,0,0], [1,0,1], [0,1,1], [1,1,0]
    ]

    all_vertices = []

    for central_vertex in central_vertices:
        other_verts = []
        for i in range(3):
            other_verts.append(
                [x if j!=i else 1-x for j, x in enumerate(central_vertex)]
            )
        all_vertices.extend(
            [
                [central_vertex]+list(comb) for comb in 
                itertools.combinations(other_verts, 2)
            ]
        )
    
    # sort these by wall
    all_vertices=torch.tensor(all_vertices)

    wall_idxs = give_wall_idx(all_vertices)

    return all_vertices, wall_idxs

In [4]:
def construct_ballroom(sphere_positions: torch.Tensor, sphere_radii: torch.Tensor, device: str ='cpu'):
    """ 
    The environment will be a 1x1x1 cube with different shaded walls
	we will insert (nonintersecting) spheres within this

    the idea will be that each triangle has a corresponding object
    which is indexed 

    For the walls of the room, this is which side its on, from 0-5

    For the balls, this is just the index of the ball
    (starting at 6)
    """
    wall_shades = torch.linspace(.1, .9, 6).to(torch.float64)
    
    # now construct all the objects 
    all_triangles = []
    all_object_idxs = []
    # first build the room
    wall_triangles, wall_idxs = make_room()
    all_triangles.append(wall_triangles)
    all_object_idxs.append(wall_idxs)
    # Now add in the spheres
    
    all_triangles, all_object_idxs = torch.concatenate(all_triangles), torch.concatenate(all_object_idxs)
        
    BallRoom = Environment(
        sphere_positions=sphere_positions.clone(), 
        sphere_radii=sphere_radii.clone(),
        triangles=all_triangles.clone(),
        object_idxs=all_object_idxs.clone(),
        device=device
    )

    BallRoom.wall_shades=wall_shades

    return BallRoom


In [5]:
class RayEnsemble:
    """ 
    Make an ensemble of rays that bounce off of spheres, and 
    get absorbed by walls.
    """
    def __init__(
            self, 
            env: Environment, 
            view_position: torch.Tensor, 
            rotation: torch.Tensor, 
            phi: float | tuple[float, float], 
            theta: float | tuple[float, float], 
            x_num: int, 
            y_num: int, 
            device: torch.device | str ='cpu',
            dtype: torch.dtype = torch.float64,
            distance_shading=True,
    ):
        self.env = env 
        self.env.to(device=device, dtype=dtype)
        self.view_position=view_position.to(dtype=torch.float64, device=torch.device(device))
        self.rotation=rotation.to(dtype=torch.float64, device=torch.device(device))
        if isinstance(phi, float):
            self.phi=(-phi, phi)
        else:
            assert len(phi)==2
            self.phi = (phi[0], phi[1]) 

        if isinstance(theta, float):
            self.theta=(-theta, theta)
        else:
            assert len(theta)==2
            self.theta = (theta[0], theta[1])  
        self.x_num=x_num 
        self.y_num=y_num
        self.device = device
        self.dtype=dtype
        self.distance_shading=distance_shading

        # now create the tensor of rays 
        vectors = []
        for phi_val in np.linspace(self.phi[0], self.phi[1], x_num):
            for theta_val in np.linspace(self.theta[0], self.theta[1], y_num):
                vectors.append(
                    [
                        np.cos(phi_val)*np.cos(theta_val),
                        np.sin(phi_val)*np.cos(theta_val),
                        np.sin(theta_val)
                    ]
                )
            
        # now apply the rotation to the vectors
        self.vectors = einops.einsum(
            rotation,
            torch.tensor(vectors).to(torch.float64),
            'i j, ... j -> ... i'
        )

        # now we need to create a tensor of values
        self.final_ray_values = torch.full(
            size = (self.vectors.shape[0],), 
            fill_value=-1,
            dtype=torch.float64, 
            device=torch.device(device)
        )

        # and keep track of rays
        self.ray_positions = torch.zeros_like(self.vectors, dtype=torch.float64, device=torch.device(device))

        # next we need to shift the environment triangles by the position
        # (allowing us to start at 0)
        self.env.triangles = self.env.triangles.clone() - self.view_position
        self.env.sphere_positions = self.env.sphere_positions.clone() - self.view_position

        # which rays are active? (not yet absorbed)
        self.unabsorbed_ray_indices = torch.arange(self.vectors.shape[0])
    
    @classmethod
    def batch_triangle_ray_trace_step(
            cls, 
            positions: torch.Tensor, 
            rays: torch.Tensor, 
            triangles: torch.Tensor
        ):
        """ 
        Solve the simple linear problem of finding the intersection 
        between a triangle (defined by 3 points) and a ray

        positions: tensor of shape (num_rays, 3)
        rays: tensor of shape (num_rays, 3)
        triangles: tensor of shape (num_triangles, 3, 3)
            corresponds to (num_triangles, three vectors, three spatial dims)
        
        return the tensor of ray multipliers:

            a tensor of shape (num_rays, num_triangles)
            This contains a float multiplier for each ray, indicating the scale multiplier 
            before it intersects the corresponding triangle 
            
            If it never does, the corresponding element is -1

        We determine this by solving the following:

        c*v = z1 + a*(z2-z1) + b*(z3-z1)

        Where we insist that 0<= a, b
        that a+b<=1
        (the ray intersects the plane in the convex hull of z1, z2, and z3)

        and that c>=0 
        (the triangle is ahead of the ray)

        Suppose each ray lives at (0,0,0)
        This is then a problem of first solving the following:

        gi := (z2-z1)_{i}
        hi := (z3-z1)_{i}

        [
            [g0, h0, -v0],
            [g1, h1, -v1],
            [g2, h2, -v2]
        ] @ [
            [a],
            [b],
            [c]
        ] = -z1
        
        Then checking that the solution obeys the constraints

        return the boolean mask (num_rays, num_triangles)
        And return the corresponding scalar which multiplies each 

        Note that introducing position only changes one aspect:
        in the equation, we essentially want to subtract off pos from each triangle vertex
        This has no effect on difference terms
        leaving us with 

        c*v = z1-pos + a*(z2-z1) + b*(z3-z1)
        """
        num_triangles, _, _ = triangles.shape 
        num_rays, _ = rays.shape
        # first, construct the tensor of matrices

        # shape = (num_rays, num_triangles, 3 vectors in a triangle, 3 dims)
        matrix_tensor = -triangles[:, :1, :].unsqueeze(0).expand(num_rays, num_triangles, 3, 3)
        matrix_tensor += triangles.unsqueeze(0).expand(num_rays, num_triangles, 3, 3)

        # now add the rays in 
        matrix_tensor[..., 0, :] -= rays.view(num_rays, 1, 3)

        solution_tensor = -triangles[:, 0, :].unsqueeze(0).expand(num_rays, num_triangles, 3)
        solution_tensor += positions.unsqueeze(1)

        # great, now we have our matrix
        # the first step is to check for zero determinants
        has_sol_mask = ~torch.isclose(torch.linalg.det(matrix_tensor), torch.tensor(0.0, dtype=torch.float64))

        # now we'll flatten, and only solve for those which have nonzero determinant 
        # we flip this (ij -> ji) so we can multiply on the left 
        matrix_tensor = einops.rearrange(matrix_tensor, "n m i j -> (n m) j i")
        has_sol_mask = einops.rearrange(has_sol_mask, "n m -> (n m)")
        solution_tensor = einops.rearrange(solution_tensor, "n m i -> (n m) i")

        # now for those with any solution, we can solve 
        solutions = torch.linalg.solve(
            matrix_tensor[has_sol_mask], 
            solution_tensor[has_sol_mask]
        )

        # Now check that it satisfies our criteria 
        # 1) 
        # that 0 <= a, b, c
        # that a + b <= 1
        good_sol_mask = (
            solutions.min(dim=-1).values>=0
        )&(
            solutions[:, 1:].sum(-1)<=1
        )&(
            solutions[:, 0]>10**-5
        ) 

        # use this to get the indices which have good solutions
        good_sol_indxs = torch.nonzero(has_sol_mask, as_tuple=False).squeeze()[good_sol_mask]
        
        # And then rebuild the boolean mask of shape (num_rays, num_triangles)
        final_sols = torch.full(size=(num_rays*num_triangles,), fill_value=-1, dtype=torch.float64)

        final_sols[good_sol_indxs] = solutions[good_sol_mask, 0]

        return final_sols.reshape((num_rays, num_triangles))

    @classmethod
    def batch_sphere_ray_trace_step(
        cls, 
        positions: torch.Tensor, 
        rays: torch.Tensor, 
        sphere_centers: torch.Tensor, 
        sphere_radii: torch.Tensor
    ):
        """ 
        Solve for the intersections with the sphere 

        Return a tensor of shape (num_rays, num_spheres)
        
        the values are either -1 (no intersection)
        or the constant multiplier giving the distance to the sphere

        Note we can solve for this easily

        We want solutions of 

        (c*v - (sphere_center-ray_position))**2 == radius**2

        Note we have shifted such that the current ray position is 0
        
        d := sphere_center-ray_position
        v.v = 1

        (c*v - d)**2 == r**2

        c**2 - 2*c*(v.d) + (d.d - r**2) == 0

        c = (2*(v.d) +/- torch.sqrt(4 * (v.d)**2 - 4 * (d.d - r**2)))/2

        c = (v.d) +/- torch.sqrt((v.d)**2 - (d.d - r**2))

        We require that c>0 (>10**-5)
        """
        num_rays, _ = rays.shape 
        num_spheres, _ = sphere_centers.shape

        # the constants we'll use to indicate how to multiply vectors to intersect spheres
        # (num_rays, num_spheres)
        constants = torch.full(size=(num_rays, num_spheres), fill_value=-1, dtype=torch.float64)
        
        # the position of the sphere centers wrt the ray positions
        # (num_rays, num_spheres, 3)
        delta = sphere_centers.unsqueeze(0)-positions.unsqueeze(1)
        
        # compute terms for the solution of c**2 - 2*c*(v.d) + (d.d - r**2) == 0
        # v.delta, (num_rays, num_spheres)
        v_dot_delta = (rays.unsqueeze(1)*delta).sum(-1)
        # discriminant (num_rays, num_spheres)
        discriminant = v_dot_delta**2 - ((delta**2).sum(-1) - sphere_radii.unsqueeze(0)**2)

        # now the following is a subtle point:
        # we want to treat these rays as internal or external
        # But how do we treat them if they are currently touching a sphere?
        # we will say that if it is touching the sphere, 
        # and v.d>0 (pointing within the sphere), then it is in the sphere.
        # Otherwise, it is outside of the sphere.

        distance_outside_radii = ((delta**2).sum(-1) - sphere_radii.unsqueeze(0)**2)

        internal_reflecting = (torch.abs(distance_outside_radii)<10**-3) & (v_dot_delta>0)

        internal_mask = (distance_outside_radii < 0) | internal_reflecting
        external_mask = (~internal_mask) & (discriminant>=0) & (v_dot_delta>0)

        # if we're outside, we expect two solutions, take v.d - sqrt ...
        # if we're inside, we expect one solution, take v.d + sqrt...

        constants[internal_mask] = (v_dot_delta + torch.sqrt(discriminant))[internal_mask]
        constants[external_mask] = (v_dot_delta - torch.sqrt(discriminant))[external_mask]
        
        constants[constants<10**-5]=-1

        return constants

    def reflect_off_spheres(self, sphere_indices):
        # this is applied only once the only vectors remaining are reflecting 
            
        assert self.vectors.shape[0] == sphere_indices.shape[0]

        normal_vectors = self.ray_positions-self.env.sphere_positions[sphere_indices]
        normal_vectors /= normal_vectors.norm(2, -1, keepdim=True)

        # now flip this component 
        self.vectors -= 2*normal_vectors*(self.vectors*normal_vectors).sum(-1, keepdim=True)

    def resolve_rays(self, sphere_indices, triangle_indices, sphere_mask, triangle_mask):
        """ 
        assign values 

        We'll multiply the current ray_value by the final shade
        along with a distance scaler 

        Then add the value to final_ray_values

        Finally, kill the elements from the vectors that have been absorbed
        """
        object_indices=self.env.object_idxs[triangle_indices]
        # get the shades 
        # note we start from 0,0,0
        wall_distances = self.ray_positions[triangle_mask].norm(2, -1)
        absorbing_objects = object_indices[triangle_mask]
        if self.distance_shading:
            shades = self.env.wall_shades[absorbing_objects]/(.1+ wall_distances/2)**(1.2)
        else:
            shades = self.env.wall_shades[absorbing_objects]
        
        # assign the final colors
        self.final_ray_values[self.unabsorbed_ray_indices[triangle_mask]] = shades

        # We only care about reflected rays
        self.vectors = self.vectors[sphere_mask]
        self.ray_positions = self.ray_positions[sphere_mask]
        self.unabsorbed_ray_indices = self.unabsorbed_ray_indices[sphere_mask]

        # reflect rays 
        self.reflect_off_spheres(
            sphere_indices[sphere_mask], 
        )
    
    def propagate_rays(self):
        """ 
        Now let's update vectors py propagating them to their closest intersecting object
        Then updating their direction with a single reflection
        """
        if not self.vectors.size:
            return None 
        
        triangle_intersection_consts = self.batch_triangle_ray_trace_step(
            positions=self.ray_positions, 
            rays=self.vectors, 
            triangles=self.env.triangles
        )

        sphere_intersection_consts = self.batch_sphere_ray_trace_step(
            positions=self.ray_positions, 
            rays=self.vectors, 
            sphere_centers=self.env.sphere_positions,
            sphere_radii=self.env.sphere_radii
        )

        num_rays, num_triangles = triangle_intersection_consts.shape
        num_rays, num_spheres = sphere_intersection_consts.shape

        triangle_intersection_consts[triangle_intersection_consts<10**-5]=float('inf')
        sphere_intersection_consts[sphere_intersection_consts<10**-5]=float('inf')

        # now we have all the multipliers 
        # we're doing either 100% opaque or reflect
        # for each ray find the closest intersection 
        triangle_hit_indices = triangle_intersection_consts.argmin(-1)
        triangle_hit_consts = triangle_intersection_consts[torch.arange(num_rays), triangle_hit_indices]

        sphere_hit_indices = sphere_intersection_consts.argmin(-1)
        sphere_hit_consts = sphere_intersection_consts[torch.arange(num_rays), sphere_hit_indices]

        hits_sphere = (triangle_hit_consts>=sphere_hit_consts) & torch.isfinite(sphere_hit_consts)
        hits_triangle = (triangle_hit_consts<sphere_hit_consts) & torch.isfinite(triangle_hit_consts)

        hit_consts = torch.minimum(sphere_hit_consts, triangle_hit_consts)

        # update_all_positions
        self.ray_positions = self.ray_positions + hit_consts.unsqueeze(-1) * self.vectors
        
        # three possible cases
        # Case 1: the ray is absorbed
        # Case 2: the ray is reflected
        # Case 3: the ray goes off to infinity
        self.resolve_rays(
            sphere_hit_indices,
            triangle_hit_indices,
            hits_sphere,
            hits_triangle
        )
    
    def display(self, filename=None, invert=False):
        fig, ax=plt.subplots(dpi=300)

        shape = (self.x_num, self.y_num)

        # format values
        output_values = self.final_ray_values.clone()
        output_values[output_values<0] = 0

        if invert:
            # we just want to show what hasnt been colored in yet 
            output_values[output_values>0] = 2.0

        output_values = output_values.reshape(shape).T

        ax.imshow(output_values, cmap='grey', vmin=0, vmax=2, origin='lower')

        aspect = (self.theta[1]-self.theta[0]) / (self.phi[1]-self.phi[0])

        aspect *= self.x_num/self.y_num

        print("aspect=",aspect)

        ax.set_aspect(aspect)
        ax.set_axis_off()
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.spines['left'].set_visible(False)
        ax.spines['bottom'].set_visible(False)

        if filename is not None:
            plt.savefig(filename, bbox_inches='tight', pad_inches=0)
            
        return fig, ax 

In [ ]:
br = construct_ballroom(
    sphere_positions=torch.tensor([[2,2,2],]), 
    sphere_radii = torch.tensor([.3]), 
    device='cpu'
)


theta = -np.radians(45)  # Convert 30 degrees to radians
rotation_matrix_z = torch.tensor([
    [np.cos(theta), -np.sin(theta), 0],
    [np.sin(theta), np.cos(theta), 0],
    [0, 0, 1]
]).to(torch.float64)

rotation_matrix_y = torch.tensor([
    [np.cos(theta), 0, -np.sin(theta)],
    [0, 1, 0],
    [np.sin(theta), 0, np.cos(theta)]
]).to(torch.float64)



re = RayEnsemble(
    env=br,   
    view_position = torch.tensor([0.1, .5, .9]), 
    rotation=rotation_matrix_y , #torch.eye(3, dtype=torch.float64), 
    phi=np.pi*.3, 
    theta=np.pi*.6/3, 
    x_num=1000, 
    y_num=1500, 
    distance_shading=False
)

for i in range(9):
    re.propagate_rays()

re.display(f"no_shading.png") 

In [ ]:
br = construct_ballroom(
    sphere_positions=torch.tensor([[2,2,2],]), 
    sphere_radii = torch.tensor([.3]), 
    device='cpu'
)


theta = -np.radians(45)  # Convert 30 degrees to radians
rotation_matrix_z = torch.tensor([
    [np.cos(theta), -np.sin(theta), 0],
    [np.sin(theta), np.cos(theta), 0],
    [0, 0, 1]
]).to(torch.float64)
theta = -np.radians(10)
rotation_matrix_y = torch.tensor([
    [np.cos(theta), 0, -np.sin(theta)],
    [0, 1, 0],
    [np.sin(theta), 0, np.cos(theta)]
]).to(torch.float64)



re = RayEnsemble(
    env=br,   
    view_position = torch.tensor([0.1, .99, .6]), 
    rotation=rotation_matrix_z@rotation_matrix_y , #torch.eye(3, dtype=torch.float64), 
    phi=np.pi*.3, 
    theta=np.pi*.6/3, 
    x_num=1000, 
    y_num=1500, 
    distance_shading=False
)

for i in range(9):
    re.propagate_rays()

re.display(f"cursed_room.png") 

In [ ]:
br = construct_ballroom(
    sphere_positions=torch.tensor([[2,2,2],]), 
    sphere_radii = torch.tensor([.3]), 
    device='cpu'
)


theta = -np.radians(30)  # Convert 30 degrees to radians
rotation_matrix_z = torch.tensor([
    [np.cos(theta), -np.sin(theta), 0],
    [np.sin(theta), np.cos(theta), 0],
    [0, 0, 1]
]).to(torch.float64)



re = RayEnsemble(
    env=br,   
    view_position = torch.tensor([0.01, .5, .5]), 
    rotation=torch.eye(3, dtype=torch.float64), 
    phi=np.pi*.3, 
    theta=np.pi*.6/3, 
    x_num=1000, 
    y_num=1500, 
    distance_shading=True
)

for i in range(9):
    re.propagate_rays()

re.display(f"with_shading.png") 

In [ ]:
br = construct_ballroom(
    sphere_positions=torch.tensor([[.5,-.1,0.5], [1.2,.5,.5], [.5,1.2,0.5]]), 
    sphere_radii = torch.tensor([.3, .3, .3]), 
    device='cpu'
)


theta = -np.radians(30)  # Convert 30 degrees to radians
rotation_matrix_z = torch.tensor([
    [np.cos(theta), -np.sin(theta), 0],
    [np.sin(theta), np.cos(theta), 0],
    [0, 0, 1]
]).to(torch.float64)



re = RayEnsemble(
    env=br,   
    view_position = torch.tensor([0.01, .9, .5]), 
    rotation=rotation_matrix_z,#torch.eye(3, dtype=torch.float32), 
    phi=np.pi*.3, 
    theta=np.pi*.6/3, 
    x_num=1000, 
    y_num=1500, 
)

In [ ]:
for i in range(9):
    re.display(f"basic_images/reflection_{i}.png") 
    re.propagate_rays()

Let's do something cool with this 

What's cooler than fractals?

In [6]:

def inverse_coords(sphere_centers, sphere_radii):
    num_spheres, _ = sphere_centers.shape
    if len( sphere_radii.shape)==2:
        sphere_radii = sphere_radii.squeeze(-1)
    inverted_coords = torch.zeros((num_spheres, 5))

    inverted_coords[..., :3] = sphere_centers / sphere_radii

    r_squared = (sphere_centers**2).sum(-1) 

    inverted_coords[..., 3] = (r_squared - sphere_radii**2 - 1) / (2*sphere_radii)
    inverted_coords[..., 4] = (r_squared - sphere_radii**2 + 1) / (2*sphere_radii)

    return inverse_coords

def give_curvature(inv_coords):
    return inv_coords[..., 4] - inv_coords[..., 3]

def standard_coords(inv_coords):
    curvatures = give_curvature(inv_coords)
    sphere_data = inv_coords[..., :3]/curvatures.unsqueeze(-1)

    sphere_data[..., -1] = 1/curvatures

    return sphere_data

In [7]:
@dataclass
class ApollonianPacking:
    """ 
    This is actually fairly simple

    each iteration has a bunch of sets of 5 spheres

    Each of these 5-sphere-sets can generate another set of 5 sphere-sets

    """
    def __init__(self):
        self.initial_basis = torch.eye(5, dtype=torch.int32)
        self.rt2o2 = math.sqrt(2) / 2
        self.rt6o2 = math.sqrt(6) / 2

        self.iteration_idx = 0

        # each generation contains the 5 spheres 
        # which define the next generation
        self.inv_sphere_generations = [
            torch.eye(5).to(dtype=torch.int32)
        ]

        self.bank = torch.eye(5).to(dtype=torch.int32)

        self.conversion_matrix = torch.tensor(
            [
                [0, -1, 1, 1, -1],
                [0, 1, -1, 1, -1],
                [0, 1, 1, -1, -1],
                [1, -1, -1, -1, -1],
                [0, 1, 1, 1, 1]
            ],
            dtype=torch.float64
        )

        self.conversion_matrix[:3, :] *= self.rt2o2
        self.conversion_matrix[4, :] *= self.rt6o2

        self.iteration_tensor = (torch.eye(5)[:, None] +  torch.eye(5)[None, :] )
        self.iteration_tensor[torch.arange(5), torch.arange(5), torch.arange(5)] -= 3
        self.iteration_tensor = self.iteration_tensor.to(dtype=torch.int32)
    
    @property
    def iterations(self) -> int:
        return len(self.inv_sphere_generations)

    def inverse_coords(self, sphere_centers, sphere_radii):
        num_spheres, _ = sphere_centers.shape
        if len( sphere_radii.shape)==2:
            sphere_radii = sphere_radii.squeeze(-1)
        inverted_coords = torch.zeros((num_spheres, 5))

        inverted_coords[..., :3] = sphere_centers / sphere_radii

        r_squared = (sphere_centers**2).sum(-1) 

        inverted_coords[..., 3] = (r_squared - sphere_radii**2 - 1) / (2*sphere_radii)
        inverted_coords[..., 4] = (r_squared - sphere_radii**2 + 1) / (2*sphere_radii)

        return inverted_coords

    def give_curvature(self, inv_coords):
        return inv_coords[..., 4] - inv_coords[..., 3]

    def standard_coords(self, inv_coords):
        curvatures = self.give_curvature(inv_coords)
        sphere_data = inv_coords[..., :4]/curvatures.unsqueeze(-1)

        sphere_data[..., -1] = 1/curvatures

        return sphere_data

    def SpecialToCartesian(self, inverted_spheres):
        assert inverted_spheres.shape[-1]==5

        transformed_inverse_coords = einops.einsum(
            self.conversion_matrix, inverted_spheres.to(dtype=torch.float64), "i j, ... j -> ... i"
        )

        return self.standard_coords(transformed_inverse_coords)
    
    def obtain_new_generation(self) -> torch.Tensor:
        """ 
        given the current spheres (in inverted coordinates)
        return the next generation.
        """
        # group for next generation
        tmp = einops.repeat(
            self.inv_sphere_generations[-1], 
            "a c -> a b c",
            b=5
        )
        
        # obtain next coordinates
        next_coords = einops.einsum(
            self.iteration_tensor.to(torch.int64),
            tmp.to(torch.int64),
            "base_sphere d1 d2, group_size base_sphere d2 -> group_size base_sphere d1"
        )

        # remove repeated coordinates
        pruned_coords = torch.unique(einops.rearrange(
            next_coords, 
            "group_size base_sphere1 d1 -> (group_size base_sphere1) d1"
        ), dim=0)

        # remove those in the bank 
        old_tensor_mask = (
            pruned_coords[:, None, :] == self.bank[None, :, :]
        ).all(-1).any(-1)

        new_generation = pruned_coords[~old_tensor_mask]

        self.bank = torch.concat(
            [self.bank, new_generation]
        )

        self.inv_sphere_generations.append(
            new_generation
        )

        return new_generation
    
    def output_all_spheres(self):
        # check for repeats, then return the spheres in 
        # their real-space coords

        # remove duplicates from each generation
        real_space_bank = self.SpecialToCartesian(self.bank)
        sphere_centers = real_space_bank[:, :3]
        sphere_radii = real_space_bank[:, -1]
        
        good_mask = (sphere_radii>0) & torch.isfinite(sphere_radii)

        sphere_centers = sphere_centers[good_mask]
        sphere_radii = sphere_radii[good_mask]

        return sphere_centers, sphere_radii
    

In [ ]:
ap.inv_sphere_generations

In [ ]:
gen_sizes

In [ ]:
# let's plot the number of spheres at each generation 

fig, ax = plt.subplots(dpi=300)

ap  = ApollonianPacking()

for i in range(7):
    ap.obtain_new_generation()

gen_sizes = [len(g) for g in ap.inv_sphere_generations]

tot_sizes = [gen_sizes[0]]
for i in range(1, len(gen_sizes)):
    tot_sizes.append(tot_sizes[-1]+gen_sizes[i])

ax.plot(range(1, 1+ len(tot_sizes)), tot_sizes)
ax.scatter(range(1, 1+ len(tot_sizes)), tot_sizes)

ax.set_title("Spheres in Apollonian Sphere Packing", fontsize=18)
ax.set_ylabel("Number of Spheres", fontsize=16)
ax.set_xlabel("Generation", fontsize=16)

ax.set_yscale("log")

In [107]:
def give_ap_spheres(gens, last_gen = False):
    ap  = ApollonianPacking()

    for i in range(gens):
        ap.obtain_new_generation()
    
    if last_gen:
        ap.bank = ap.inv_sphere_generations[-1]

    final_spheres, final_radii = ap.output_all_spheres()
    all_radii = torch.sqrt((final_spheres**2).sum(-1)) + final_radii + .2

    final_spheres /= 2*all_radii.unsqueeze(-1)

    final_radii /= 2*all_radii

    final_spheres += torch.tensor([[0.5, 0.5, 0.5]])

    return final_spheres, final_radii

I want a banner image

Ill make a high-res one 

In [108]:
gens=5

In [ ]:
final_spheres, final_radii = give_ap_spheres(gens=gens, last_gen=True)

br = construct_ballroom(
    sphere_positions=final_spheres, 
    sphere_radii = final_radii, 
    device='cpu'
)
print(br.sphere_positions)
print(br.sphere_radii)

theta = np.radians(-30)  # Convert 30 degrees to radians
rotation_matrix_y = torch.tensor([
    [np.cos(theta), 0, -np.sin(theta)],
    [0,1,0],
    [np.sin(theta), 0, np.cos(theta)]
]).to(torch.float64)

theta = np.radians(30)  # Convert 30 degrees to radians
rotation_matrix_z = torch.tensor([
    [np.cos(theta), -np.sin(theta), 0],
    [np.sin(theta), np.cos(theta), 0],
    [0, 0, 1]
]).to(torch.float64)

tot_matrix = rotation_matrix_y@rotation_matrix_z

xnum=3_000
ynum=3_000

total_output = np.zeros((xnum, ynum))

slices = 100

theta_intervals = []
endpoints = np.linspace(-np.pi*.6/3, np.pi*.6/3, slices+1)
for i in range(slices):
    theta_intervals.append(
        (endpoints[i], endpoints[i+1])
    )

for i in range(slices):
    print(f"starting slice {i}")
    final_spheres, final_radii = give_ap_spheres(gens=gens, last_gen=True)

    br = construct_ballroom(
        sphere_positions=final_spheres, 
        sphere_radii = final_radii, 
        device='cpu'
    )

    re = RayEnsemble(
        env=br,   
        view_position = torch.tensor([0.01, .2, .8]), 
        rotation=tot_matrix, #torch.eye(3, dtype=torch.float64), 
        phi=np.pi*.3, 
        theta=theta_intervals[i], 
        x_num=xnum, 
        y_num=ynum//slices, 
    )

    invert=False

    for j in range(1000):
        
        re.propagate_rays()
    
    shape = (re.x_num, re.y_num)

    # format values
    output_values = re.final_ray_values.clone()
    output_values[output_values<0] = 0

    if invert:
        # we just want to show what hasnt been colored in yet 
        output_values[output_values>0] = 2.0

    output_values = output_values.reshape(shape)

    print("output_values.shape",output_values.shape)

    print(i*(ynum//slices), (i+1)*(ynum//slices))

    total_output[:, i*(ynum//slices):(i+1)*(ynum//slices)] = output_values

    print(f"finishing slice {i}")
    print("")




In [ ]:
final_spheres.mean(0)

In [ ]:
total_output

In [ ]:
total_output

In [ ]:
fig, ax = plt.subplots(dpi=1_000)

ax.imshow(total_output.T, cmap='grey', vmin=0, vmax=2, origin='lower')

aspect = (2*endpoints[-1]) / (2*np.pi*.3)

aspect *= xnum/ynum

print("aspect=",aspect)

ax.set_aspect(aspect)
ax.set_axis_off()


print("aspect=",aspect)

ax.set_aspect(aspect)
ax.set_axis_off()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(False)

plt.tight_layout()

plt.savefig("last_gen_6_Banner.png", bbox_inches='tight', pad_inches=0)
    


In [ ]:
def give_zoom_pic(z, re= None, target_phi_frac=None, target_theta_frac=None, zoom_at_target=None):
    if re is None:

        ap  = ApollonianPacking()

        final_spheres, final_radii = ap.output_all_spheres()
        all_radii = torch.sqrt((final_spheres**2).sum(-1)) + final_radii + .2

        final_spheres /= 2*all_radii.unsqueeze(-1)

        final_spheres += torch.tensor([[.5, .5, .5]])

        final_radii /= 2*all_radii

        br = construct_ballroom(
            sphere_positions=final_spheres, 
            sphere_radii = final_radii, 
            device='cpu'
        )
        print(br.sphere_positions)
        print(br.sphere_radii)

        theta = np.radians(-30)  # Convert 30 degrees to radians
        rotation_matrix_y = torch.tensor([
            [np.cos(theta), 0, -np.sin(theta)],
            [0,1,0],
            [np.sin(theta), 0, np.cos(theta)]
        ]).to(torch.float64)

        theta = np.radians(30)  # Convert 30 degrees to radians
        rotation_matrix_z = torch.tensor([
            [np.cos(theta), -np.sin(theta), 0],
            [np.sin(theta), np.cos(theta), 0],
            [0, 0, 1]
        ]).to(torch.float64)

        tot_matrix = rotation_matrix_y@rotation_matrix_z

        # current_center_theta -0.008892884642861063
        # current_center_phi -0.03598221109224081

        current_center_phi = np.float64(-0.03598221109224081)
        current_center_theta = np.float64(-0.008892884642861063)

        if target_phi_frac is None:
            new_phi = (current_center_phi + np.float64(-0.9424777960769379)/z, current_center_phi + np.float64(0.9424777960769379)/z)
        else:
            """
            we want to be zooming into an interesting target point 
            ie we want the fraction of the phi where our interesting thing is to be .5
            0 = current_center_phi + delta + (target_phi_frac*2*const./z - const./z)
            and we have current_center_phi=0

            But this isnt going to work perfectly
            instead, we'll have to correct, and repeatedly update current_center_phi

            when z was target zoom, we had to be at a certain fraction in the image
            delta = (target_phi_frac*2*const./z - const./z)
            """
            const = np.float64(-0.9424777960769379)

            current_center_phi += (target_phi_frac*2 - 1)*const / zoom_at_target

            new_phi = (current_center_phi + np.float64(-0.9424777960769379)/z, current_center_phi + np.float64(0.9424777960769379)/z)
        
        if target_theta_frac is None:
            new_theta = (current_center_theta + np.float64(-0.6283185307179586)/z, current_center_theta + np.float64(0.6283185307179586)/z)
        else: 
            const = np.float64(-0.6283185307179586)

            current_center_theta += (target_theta_frac*2 - 1)*const / zoom_at_target

            new_theta = (current_center_theta + np.float64(-0.6283185307179586)/z, current_center_theta + np.float64(0.6283185307179586)/z)


        re = RayEnsemble(
            env=br,   
            view_position = torch.tensor([0.01, .2, .8]), 
            rotation=tot_matrix, #torch.eye(3, dtype=torch.float64), 
            phi=new_phi, 
            theta=new_theta, 
            x_num=1500, 
            y_num=1500, 
        )

    """ 
    stop zooming when doing one more iteration 
    changes the fraction still reflecting by < threshold
    """
    cur = (re.final_ray_values<0).sum()
    count=0
    while count<10 or cur / re.final_ray_values.size()[0] > .05:
        #print(cur / re.final_ray_values.size()[0])
        
        re.propagate_rays()
        cur = (re.final_ray_values<0).sum()
        count+=1

        if count >25:
            break
    
    shape = (re.x_num, re.y_num)

    # format values
    output_values = re.final_ray_values.clone()
    output_values[output_values<0] = 0

    output_values = output_values.reshape(shape).T

    np.save(f"serpinski_zoom/serpinski_zoom_{z}_inverted_app.npy", output_values)

    re.display(f"serpinski_zoom/serpinski_zoom_{z}_inverted_app.png", invert=True) 

    print("current_center_theta", current_center_theta)
    print("current_center_phi", current_center_phi)

In [ ]:
ap.inv_sphere_generations

In [ ]:
invert=False
num_gens = len(ap.inv_sphere_generations)

for i in range(18):
    
    re.propagate_rays()

    if invert:
        re.display(f"fractal_folder/num_gens_{num_gens}_inverted_app_{i}.png", invert=True) 
    else:
        re.display(f"fractal_folder/num_gens_{num_gens}_app_{i}.png", invert=False) 

In [ ]:
br = construct_ballroom(
    sphere_positions=final_spheres, 
    sphere_radii = final_radii, 
    device='cpu'
)
print(br.sphere_positions)
print(br.sphere_radii)

theta = np.radians(-30)  # Convert 30 degrees to radians
rotation_matrix_y = torch.tensor([
    [np.cos(theta), 0, -np.sin(theta)],
    [0,1,0],
    [np.sin(theta), 0, np.cos(theta)]
]).to(torch.float64)

theta = np.radians(30)  # Convert 30 degrees to radians
rotation_matrix_z = torch.tensor([
    [np.cos(theta), -np.sin(theta), 0],
    [np.sin(theta), np.cos(theta), 0],
    [0, 0, 1]
]).to(torch.float64)

tot_matrix = rotation_matrix_y@rotation_matrix_z



re = RayEnsemble(
    env=br,   
    view_position = torch.tensor([0.01, .2, .8]), 
    rotation=tot_matrix, #torch.eye(3, dtype=torch.float64), 
    phi=np.pi*.3, 
    theta=np.pi*.6/3, 
    x_num=1500, 
    y_num=1500, 
)

In [ ]:
invert=True
num_gens = len(ap.inv_sphere_generations)

for i in range(18):
    
    re.propagate_rays()

    if invert:
        re.display(f"fractal_folder/num_gens_{num_gens}_inverted_app_{i}.png", invert=True) 
    else:
        re.display(f"fractal_folder/num_gens_{num_gens}_app_{i}.png", invert=False) 

Let's zoom into the serpinski triangle and call it quits 

In [22]:
def give_zoom_pic(z, re= None, target_phi_frac=None, target_theta_frac=None, zoom_at_target=None):
    if re is None:

        ap  = ApollonianPacking()

        final_spheres, final_radii = ap.output_all_spheres()
        all_radii = torch.sqrt((final_spheres**2).sum(-1)) + final_radii + .2

        final_spheres /= 2*all_radii.unsqueeze(-1)

        final_spheres += torch.tensor([[.5, .5, .5]])

        final_radii /= 2*all_radii

        br = construct_ballroom(
            sphere_positions=final_spheres, 
            sphere_radii = final_radii, 
            device='cpu'
        )
        print(br.sphere_positions)
        print(br.sphere_radii)

        theta = np.radians(-30)  # Convert 30 degrees to radians
        rotation_matrix_y = torch.tensor([
            [np.cos(theta), 0, -np.sin(theta)],
            [0,1,0],
            [np.sin(theta), 0, np.cos(theta)]
        ]).to(torch.float64)

        theta = np.radians(30)  # Convert 30 degrees to radians
        rotation_matrix_z = torch.tensor([
            [np.cos(theta), -np.sin(theta), 0],
            [np.sin(theta), np.cos(theta), 0],
            [0, 0, 1]
        ]).to(torch.float64)

        tot_matrix = rotation_matrix_y@rotation_matrix_z

        # current_center_theta -0.008892884642861063
        # current_center_phi -0.03598221109224081

        current_center_phi = np.float64(-0.03598221109224081)
        current_center_theta = np.float64(-0.008892884642861063)

        if target_phi_frac is None:
            new_phi = (current_center_phi + np.float64(-0.9424777960769379)/z, current_center_phi + np.float64(0.9424777960769379)/z)
        else:
            """
            we want to be zooming into an interesting target point 
            ie we want the fraction of the phi where our interesting thing is to be .5
            0 = current_center_phi + delta + (target_phi_frac*2*const./z - const./z)
            and we have current_center_phi=0

            But this isnt going to work perfectly
            instead, we'll have to correct, and repeatedly update current_center_phi

            when z was target zoom, we had to be at a certain fraction in the image
            delta = (target_phi_frac*2*const./z - const./z)
            """
            const = np.float64(-0.9424777960769379)

            current_center_phi += (target_phi_frac*2 - 1)*const / zoom_at_target

            new_phi = (current_center_phi + np.float64(-0.9424777960769379)/z, current_center_phi + np.float64(0.9424777960769379)/z)
        
        if target_theta_frac is None:
            new_theta = (current_center_theta + np.float64(-0.6283185307179586)/z, current_center_theta + np.float64(0.6283185307179586)/z)
        else: 
            const = np.float64(-0.6283185307179586)

            current_center_theta += (target_theta_frac*2 - 1)*const / zoom_at_target

            new_theta = (current_center_theta + np.float64(-0.6283185307179586)/z, current_center_theta + np.float64(0.6283185307179586)/z)


        re = RayEnsemble(
            env=br,   
            view_position = torch.tensor([0.01, .2, .8]), 
            rotation=tot_matrix, #torch.eye(3, dtype=torch.float64), 
            phi=new_phi, 
            theta=new_theta, 
            x_num=1500, 
            y_num=1500, 
        )

    """ 
    stop zooming when doing one more iteration 
    changes the fraction still reflecting by < threshold
    """
    cur = (re.final_ray_values<0).sum()
    count=0
    while count<10 or cur / re.final_ray_values.size()[0] > .05:
        #print(cur / re.final_ray_values.size()[0])
        
        re.propagate_rays()
        cur = (re.final_ray_values<0).sum()
        count+=1

        if count >25:
            break
    
    shape = (re.x_num, re.y_num)

    # format values
    output_values = re.final_ray_values.clone()
    output_values[output_values<0] = 0

    output_values = output_values.reshape(shape).T

    np.save(f"serpinski_zoom/serpinski_zoom_{z}_inverted_app.npy", output_values)

    re.display(f"serpinski_zoom/serpinski_zoom_{z}_inverted_app.png", invert=True) 

    print("current_center_theta", current_center_theta)
    print("current_center_phi", current_center_phi)

In [ ]:
for z in (1.5 ** np.arange(60)):
    give_zoom_pic(z, re= None)

In [2]:
import glob 

In [3]:
def plot_zoomed_image(output_values, filename, frac):
    aspect=2/3
    
    # Calculate the indices for cropping
    ny, nx = output_values.shape
    xmin = int((1 - frac) * nx / 2)
    xmax = int((1 + frac) * nx / 2)
    ymin = int((1 - frac) * ny / 2)
    ymax = int((1 + frac) * ny / 2)

    # Crop the array to zoom in
    output_values = output_values[ymin:ymax, xmin:xmax]

    # Plot the zoomed-in image
    fig, ax = plt.subplots()
    ax.imshow(output_values, cmap='grey', vmin=0, vmax=2, origin='lower')
    ax.set_aspect(aspect)

    # Remove axes and spines as before
    ax.set_axis_off()
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.spines['bottom'].set_visible(False)

    # Save the figure at high resolution
    plt.savefig(filename, dpi=300, bbox_inches='tight', pad_inches=0)
    plt.close(fig)

In [4]:
def make_video_frames(z=1.5, frames_per_zoom=24, invert=False):

    def current_zoom(file):
        i1 = file.rindex("_inverted_app")
        i0 = (file[:i1].rindex("_") + 1)
        return float(file[i0:i1])

    files = sorted(glob.glob("serpinski_zoom/*_inverted_app.npy"), key = current_zoom)

    for file in files:
        current_tot_zoom = current_zoom(file)
        print("current_zoom = ",current_tot_zoom)

        zoom_fractions = 1 / (1.5**np.linspace(0, 1, frames_per_zoom+1)[:-1])

        if invert:
            file_names = [f"zoom_video_files/inverted_zoom_video_files_{current_tot_zoom*cz}.png" for cz in (1.5**np.linspace(0, 1, frames_per_zoom+1)[:-1])]
        else:
            file_names = [f"zoom_video_files/zoom_video_files_{current_tot_zoom*cz}.png" for cz in (1.5**np.linspace(0, 1, frames_per_zoom+1)[:-1])]

        output_values = np.load(file)

        if invert:
            # we just want to show what hasnt been colored in yet 
            output_values[output_values>0] = 2.0

        for file_name, frac in zip(file_names, zoom_fractions):
            plot_zoomed_image(output_values, file_name, frac)

In [ ]:
make_video_frames()

In [11]:
from moviepy.editor import ImageSequenceClip

def create_video_from_images(image_files, output_path, fps=24, bitrate='8000k', resolution=None):
    """
    Create a high-quality video from a list of PNG images.

    Args:
        image_files (list of str): List of PNG image file paths.
        output_path (str): Path where the output MP4 file will be saved.
        fps (int, optional): Frames per second for the video. Defaults to 24.
        bitrate (str, optional): Video bitrate (e.g., '5000k' for 5000 kbps). Adjust to balance quality and file size.
        resolution (tuple, optional): Desired resolution as (width, height). If None, uses original image sizes.
    """

    def current_zoom(file):
        i0 = file.rindex("_")+1
        i1 = -4
        return float(file[i0:i1])

    image_files = sorted(image_files, key = current_zoom)
    

    if not image_files:
        raise ValueError("The list of image files is empty. Please provide valid image file paths.")

    # Create an image sequence clip
    clip = ImageSequenceClip(image_files, fps=fps)

    # Check if clip has valid duration
    if clip.duration is None:
        raise ValueError("The clip duration is None. Please check the images and fps value.")

    # Resize clip if resolution is specified
    if resolution is not None:
        clip = clip.resize(newsize=resolution)

    # Write the video file with specified codec and bitrate
    clip.write_videofile(
        output_path,
        codec='libx264',        # H.264 codec for MP4
        bitrate=bitrate,        # Controls quality and file size
        audio=False,            # No audio
        threads=4,              # Speeds up the processing
        preset='veryslow'         # Balance between encoding speed and compression
    )


In [ ]:
create_video_from_images(glob.glob('zoom_video_files/zoom_video_files*.png'), 'test_video.mp4')

In [ ]:
make_video_frames(invert=True)

In [ ]:
create_video_from_images(glob.glob('zoom_video_files/inverted_zoom_video_files*.png'), 'test_video_inverted.mp4')